In [1]:
from pre_processing import *
from lda_mallet import *

from os import walk


#reading data
datafolder = './data/ed_recovery_formatted/Excel'
#datafolder = 'ed_recovery_topicmodel'
df = pd.DataFrame(columns=['url','type','photo','date','tags','notes','text','photo_url','reblogged','blog_name'])

for dirpath,dirnames,filenames in walk(datafolder):
  for filename in filenames:
    if filename.endswith('.xlsx'):
      t = pd.read_excel(datafolder+'/'+filename,names=['url','type','photo','date','tags','notes','text','photo_url','reblogged'])
      blog_name = t.iloc[0,0].split(':')[1]
      t['blog_name'] = blog_name
      df = df.append(t.iloc[3:,:],ignore_index=True)
      print('blog:{0}   with posts:{1}    and reblogs:{2}  '.format(filename,len(t),len(t[t.reblogged=='yes'])))


print('number of blogs: {0} - number of posts: {1}'.format(len(set(df.blog_name)),len(df)))
print('out of {0} documents, {1} are reblogged.'.format(len(df),len(df[df.reblogged == 'yes'])))

#finding reblogged texts
texts = sorted(df.text) #sort them to keep smallest post (perhaps original one) at first
re_texts = []

while len(texts):
  t = [texts.pop(0)]#pop first text and find it!
  if t[0] == ' ' or len(t[0].split())<3: 
    continue #almost nothing to look
  i = 0
  while i<len(texts):
    if t[0] in texts[i]:
      t.append(texts.pop(i))
    else:
      i += 1
  if len(t) > 1:
    re_texts.append(t)


print('number of unique reblogged texts: {0}'.format(len(re_texts)))
print('number of unique string in all texts: {0}'.format(len(set(df.text))))

extra_stopwords = ['isnt','want','cant','wanna','im','could','ive','would','dont','get','also','us','thats','got','ur','wanted',
                   'may', 'the', 'just', 'can', 'think', 'damn', 'still', 'guys', 'literally', 'hopefully', 'much', 'even', 'rly', 'guess', 'anon']#anything with a length of one
                   

'''pre-processing'''
original_doc_set = list(df.text[df.photo=='no'])
pre_processed_docs,filtered_docs = preprocess_data(original_doc_set,extra_stopwords=extra_stopwords)
vocab_dict, doc_term_matrix = prepare_corpus(pre_processed_docs)

print('size of orginal dataset: {0} and size of the pre-processed dataset: {1}'.format(len(original_doc_set),len(pre_processed_docs)))

pre_processed_docs_lem,filtered_docs_lem = preprocess_data(original_doc_set,extra_stopwords=extra_stopwords,lemmatized=True)
vocab_dict_lem, doc_term_matrix_lem = prepare_corpus(pre_processed_docs_lem)

print('Vocab size before lemmatiziation: {0} and after lemmatization: {1}'.format(len(vocab_dict),len(vocab_dict_lem)))


[nltk_data] Downloading package stopwords to /Users/das-
[nltk_data]     lab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/das-lab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/das-lab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/das-lab/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


blog:mote-of-dust_formatted.xlsx   with posts:271    and reblogs:238  
blog:fuck-eatingdisorders_formatted.xlsx   with posts:37    and reblogs:4  
blog:hellsite-residential_formatted.xlsx   with posts:247    and reblogs:191  
blog:lets-get-better_formatted.xlsx   with posts:509    and reblogs:453  
blog:bagelbells_formatted.xlsx   with posts:1101    and reblogs:1046  
blog:imtrying-butimpissed_formatted.xlsx   with posts:838    and reblogs:483  
blog:faithhopeloveandtherapy_formatted.xlsx   with posts:2994    and reblogs:1510  
blog:finallyrecoveringforgood_formatted.xlsx   with posts:137    and reblogs:49  
blog:intentandotrascender_formatted.xlsx   with posts:21    and reblogs:0  
blog:fearless-foodie_formatted.xlsx   with posts:3956    and reblogs:3670  
blog:ariessuntaurusrising_formatted.xlsx   with posts:504    and reblogs:206  
blog:rec-hovery_formatted.xlsx   with posts:22    and reblogs:7  
blog:shameofateen_formatted.xlsx   with posts:54    and reblogs:2  
blog:brittle-bunny_

In [1]:
from pre_processing import *
#loading the dataset we train the model on
text_df = newsgroup('./data/20newsgroup_preprocessed.csv')
doc_list = list(text_df.text_cleaned)
pre_processed_docs,filtered_docs = preprocess_data(doc_list)
vocab_dict_, doc_term_matrix_ = prepare_corpus(pre_processed_docs)

[nltk_data] Downloading package stopwords to /Users/das-
[nltk_data]     lab/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/das-lab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/das-lab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/das-lab/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
# vocab_dict_['existence']
vocab_dict_.token2id['ax']

13588

In [5]:
#loading reference corpus
wiki_docs = loading_wiki_docs('./data/wiki_sampled_5p.txt')
pre_processed_wiki,no_var = preprocess_data(wiki_docs)
vocab_dict, doc_term_matrix = prepare_corpus(pre_processed_wiki)

#loading the dataset we train the model on
text_df = newsgroup('./data/20newsgroup_preprocessed.csv')
doc_list = list(text_df.text_cleaned)
pre_processed_docs,filtered_docs = preprocess_data(doc_list)
vocab_dict_, doc_term_matrix_ = prepare_corpus(pre_processed_docs)

In [39]:
from lda_mallet import *

def reading_results(res,topic_num,itreations):
  '''
  Reading result to get top terms and log-likelihood every 50 iterations

  returns: Log-Likelihood and all_top terms for each 50 iterations

  parameter res: all the command prompt prints and results (type:str)
  '''

  #reading the string
  current_line = 9#where to start in CSV file
  stp1 = topic_num+2#where to read Log-Likelihood
  stp2 = 6 #constant number of lines to get to topics again

  #please set the number of iteration to what you set for your topic model
  all_top_terms = []#storing all top terms
  LLs = []
  for _i in range(int(itreations/50)):
    top_terms = []
    for i in range(topic_num):#reading top 
      try:
        #reading top terms splited by two tabs + the second split is for splitting the terms with space
        #excluding the last item using [0:-1].  because the last item is '\n'
        top_terms.append(res[current_line].split('\t')[2].split(' ')[0:-1])
      except:
        print(res[current_line-2:current_line+3],current_line)
      current_line+=1#going to next line
    current_line+=1 #going to LL
    if _i>3: #optimizing alpha would add [beta] update after 250 iterations and we want to add one line for that
      current_line +=1
    LLs.append(float(res[current_line].split(': ')[1]))
    current_line += stp2
    all_top_terms.append(top_terms)

  return all_top_terms,LLs

In [42]:
with open('t.csv','r') as csvfile:
    res = csvfile.readlines()
#   res = res.split('\n')
all_top_terms,LLs = reading_results(res,3,250)
coherence = []

In [54]:
vocab_dict.token2id['ax']
# all_top_terms

18022

In [48]:
for top_terms in all_top_terms:
    coherence.append(CoherenceModel(topics=top_terms,dictionary=vocab_dict,texts=pre_processed_wiki,topn=5,coherence='c_npmi').get_coherence())

In [58]:
c = CoherenceModel.for_topics(topics_as_topn_terms=all_top_terms,dictionary=vocab_dict,texts=pre_processed_wiki,topn=5,coherence='c_npmi')